In [2]:
import os
from google.cloud import bigquery
from google.cloud.bigquery import ScalarQueryParameter, QueryJobConfig
import pandas as pd

# 認証情報のパスを設定
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/atsuk/OneDrive/ドキュメント/GeminiCLI/2507 PatentsFinder/corded-guild-459506-i6-5bc162f91e16.json"

client = bigquery.Client()

# 検索条件
ipc_codes = ['B01D 61/00', 'G06N 20/00']
keywords = ['reverse osmosis membrane', 'machine learning', 'water filtration', 'membrane technology']
publication_date_from = '20160101'
# publication_date_toは指定なし（最大値で設定）
publication_date_to = '20250710'

# IPCコード条件
ipc_conditions = []
for i, ipc in enumerate(ipc_codes):
    ipc_conditions.append(f"EXISTS (SELECT 1 FROM UNNEST(p.ipc) AS ipc WHERE ipc.code LIKE @ipc_code_{i})")
ipc_sql = " OR ".join(ipc_conditions) if ipc_conditions else "1=1"

# キーワード条件
keyword_conditions = []
for i, kw in enumerate(keywords):
    keyword_conditions.append(f"LOWER((SELECT text FROM UNNEST(p.title_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1)) LIKE @keyword_{i}")
    keyword_conditions.append(f"LOWER((SELECT text FROM UNNEST(p.abstract_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1)) LIKE @keyword_{i}")
    keyword_conditions.append(f"LOWER((SELECT text FROM UNNEST(p.claims_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1)) LIKE @keyword_{i}")
keyword_sql = " OR ".join(keyword_conditions) if keyword_conditions else "1=1"

# IPCまたはキーワードのどちらかに一致
where_sql = f"({ipc_sql}) OR ({keyword_sql})"

sql = f"""
    SELECT
        p.publication_number,
        (SELECT text FROM UNNEST(p.title_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as title,
        (SELECT text FROM UNNEST(p.abstract_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as abstract,
        (SELECT text FROM UNNEST(p.claims_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as claims,
        (SELECT STRING_AGG(name) FROM UNNEST(p.assignee_harmonized)) as assignee,
        p.publication_date,
        (SELECT STRING_AGG(code) FROM UNNEST(p.ipc)) as ipc_codes
    FROM
        `patents-public-data.patents.publications` AS p
    WHERE
        p.publication_date >= @publication_date_from
        AND p.publication_date <= @publication_date_to
        AND ({where_sql})
    LIMIT 100
"""

query_params = [
    ScalarQueryParameter("publication_date_from", "INT64", int(publication_date_from)),
    ScalarQueryParameter("publication_date_to", "INT64", int(publication_date_to)),
]
for i, ipc_code in enumerate(ipc_codes):
    query_params.append(ScalarQueryParameter(f"ipc_code_{i}", "STRING", f"{ipc_code}%"))
for i, kw in enumerate(keywords):
    query_params.append(ScalarQueryParameter(f"keyword_{i}", "STRING", f"%{kw.lower()}%"))

job_config = QueryJobConfig(query_parameters=query_params)

print("--- BigQuery実行クエリ ---")
print(sql)
print("--- クエリパラメータ ---")
print(query_params)

query_job = client.query(sql, job_config=job_config)
df = query_job.result().to_dataframe()
print(f"{len(df)}件の特許が見つかりました。")
print(df.head())


--- BigQuery実行クエリ ---

    SELECT
        p.publication_number,
        (SELECT text FROM UNNEST(p.title_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as title,
        (SELECT text FROM UNNEST(p.abstract_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as abstract,
        (SELECT text FROM UNNEST(p.claims_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as claims,
        (SELECT STRING_AGG(name) FROM UNNEST(p.assignee_harmonized)) as assignee,
        p.publication_date,
        (SELECT STRING_AGG(code) FROM UNNEST(p.ipc)) as ipc_codes
    FROM
        `patents-public-data.patents.publications` AS p
    WHERE
        p.publication_date >= @publication_date_from
        AND p.publication_date <= @publication_date_to
        AND ((EXISTS (SELECT 1 FROM UNNEST(p.ipc) AS ipc WHERE ipc.code LIKE @ipc_code_0) OR EXISTS (SELECT 1 FROM UNNEST(p.ipc) AS ipc WHERE ipc.code LIKE @ipc_code_1)) OR (LOWER((SELECT text FROM UNNEST(p.title_localized) WHERE lan

c:\Users\atsuk\OneDrive\ドキュメント\GeminiCLI\2507 PatentsFinder\venv\Lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


100件の特許が見つかりました。
  publication_number                                              title  \
0   US-2019052597-A1           Optimizing choice of networking protocol   
1   US-2019199654-A1                           Network interface device   
2   US-2019220342-A1  Traffic and geography based cognitive disaster...   
3   US-2019270038-A1  Water filtration apparatus with top-loading fi...   
4   US-2019318240-A1  Training machine learning models in distribute...   

                                            abstract  \
0  Network performance data metrics are gathered ...   
1  A network interface device has an interface co...   
2  In a system having at least two data storage a...   
3  An apparatus for filtering water includes a tu...   
4  Certain aspects of the present disclosure prov...   

                                              claims  \
0  What is claimed is: \n     \n         1 . A me...   
1  1 . A network interface device comprising:\n a...   
2  What is claimed is: \n  

In [3]:
df

,publication_number,title,abstract,claims,assignee,publication_date,ipc_codes
0,US-2019052597-A1,Optimizing choice of networking protocol,Network performance data metrics are gathered ...,What is claimed is: \n \n 1 . A me...,SALESFORCE COM INC,20190214,"H04L29/12,G06N99/00,H04L12/24"
1,US-2019199654-A1,Network interface device,A network interface device has an interface co...,1 . A network interface device comprising:\n a...,SOLARFLARE COMMUNICATIONS INC,20190627,"G06N99/00,H04L45/24"
2,US-2019220342-A1,Traffic and geography based cognitive disaster...,In a system having at least two data storage a...,What is claimed is: \n \n 1 . A me...,IBM,20190718,"G06F11/07,G06N99/00,G06N5/02"
3,US-2019270038-A1,Water filtration apparatus with top-loading fi...,An apparatus for filtering water includes a tu...,"1 . An apparatus for filtering water, the appa...",FILTER GROUP INC,20190905,"B01D15/00,C02F1/00,B01D35/30,B01D36/00,B01D27/..."
4,US-2019318240-A1,Training machine learning models in distribute...,Certain aspects of the present disclosure prov...,What is claimed is: \n \n 1 . A me...,KAZUHM INC,20191017,"G06N3/04,G06K9/62,G06N3/08,G06F9/455"
...,...,...,...,...,...,...,...
95,CN-113051836-B,Rapid modeling method for antenna under machin...,The invention discloses a quick modeling metho...,None,None,20240426,"G06F30/27,G06F119/20"
96,US-10010255-B2,Systems and methods for estimating ischemia an...,Systems and methods are disclosed for determin...,What is claimed is: \n \n 1. A metho...,HEARTFLOW INC,20180703,"A61B5/024,A61B5/145,A61B6/03,G06F19/00,G06N99/..."
97,US-10136826-B2,System and method for distinguishing a cardiac...,A cardiac monitoring device includes: at least...,The invention claimed is: \n \n 1. A...,ZOLL MEDICAL CORP,20181127,"A61N1/04,G16H40/67,A61N1/39,A61B5/00,A61B5/308..."
98,US-10148819-B2,Team-based customer care routing,A user identifier of a customer may be obtaine...,What is claimed is: \n \n 1. One or ...,T MOBILE USA INC,20181204,"H04M3/523,H04M3/42,H04M3/51"
